In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from math import ceil, floor
from scipy.optimize import minimize
from tqdm import tqdm
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from cluster_local import ClusterLensing_fyp
from astropy.cosmology import FlatLambdaCDM
import pandas as pd

In [2]:
# inject data

scenarios = {
    '1': 'abell370',
    '2': 'abell2744',
    '3': 'abells1063',
    '4': 'macs0416',
    '5': 'macs0717',
    '6': 'macs1149'
}

full_cluster_names = {
    'abell370': 'Abell 370',
    'abell2744': 'Abell 2744',
    'abells1063': 'Abell S1063',
    'macs0416': 'MACS J0416.1-2403',
    'macs0717': 'MACS J0717.5+3745',
    'macs1149': 'MACS J1149.5+2223'
}

# Initialize lists to store the data arrays
datax_list = []
datay_list = []
data_psi_list = []

for i in scenarios:
    clustername = scenarios[i]
    full_cluster_name = full_cluster_names[clustername]

    file_dir = os.getcwd()
    fits_filex = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_x-arcsec-deflect.fits'
    )
    fits_filey = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_y-arcsec-deflect.fits'
    )
    psi_file = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_psi.fits'
    )

    with fits.open(fits_filex) as hdulx, fits.open(fits_filey) as hduly, fits.open(psi_file) as hdul_psi:
        datax = hdulx[0].data
        datay = hduly[0].data
        data_psi = hdul_psi[0].data

        # Append the data arrays to the lists
        datax_list.append(datax)
        datay_list.append(datay)
        data_psi_list.append(data_psi)

# Example: Access the first datax array
print(f"Length of the first datax array: {len(datax_list[2])}")


Length of the first datax array: 799


In [3]:
# getting the pixel scale list
def get_pixscale(cluster_name, file_path='GCdata/pixsize'):
    full_path = os.path.join(file_dir, file_path)
    with open(full_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith(cluster_name):
                # Split the line to get the value after the colon and return it as a float
                return float(line.split(':')[1].strip())
    return None  # Return None if the cluster name isn't found

pixscale_list = []
for i in scenarios:
    clustername = scenarios[i]
    full_cluster_name = full_cluster_names[clustername]
    pixscale = get_pixscale(full_cluster_name)
    pixscale_list.append(pixscale)
print(pixscale_list)


[0.2, 0.25, 0.25, 0.2, 0.2, 0.2]


In [4]:
# set up the class
cluster = ClusterLensing_fyp(datax_list, datay_list, data_psi_list, 0.5, 1, pixscale_list, diff_z=False)


In [5]:
print(len(datax_list[5]))

698


In [6]:
# check with the lenstronomy package directly
realsize = 698
grid = np.linspace(0, realsize-1, realsize)
grid = grid * 0.2

lens_model_list2 = ['INTERPOL']
kwargs_lens2 = [{'grid_interp_x': grid, 'grid_interp_y': grid, 'f_': data_psi_list[5]*0.2**2,
                          'f_x': datax_list[5], 'f_y': datay_list[5]}]

lensModel_real = LensModel(lens_model_list=lens_model_list2, z_source=1, z_lens=0.5)

# Use lens equation solver for verification
solver2 = LensEquationSolver(lensModel_real)
coord = (76.70065971846965, 64.7852180985455)
img_po = solver2.image_position_from_source(coord[0], coord[1], kwargs_lens2, min_distance=0.2, search_window=100, verbose=False, x_center=70, y_center=65)
print(img_po)
mag = lensModel_real.magnification(img_po[0], img_po[1], kwargs_lens2)
print(mag)
dt = lensModel_real.arrival_time(img_po[0], img_po[1], kwargs_lens2)
dt = [dt_ - dt[0] for dt_ in dt]
print(dt)

(array([106.51129008,  54.373867  ,  94.50298699,  65.15280208,
        83.72098886]), array([37.60187869, 97.69887162, 88.69469174, 59.78233641, 73.23005261]))
[ 1.74328197  1.63793269 -1.9422805  -5.59530548  0.62218531]
[0.0, 32320.47736167349, 59054.80822375999, 64898.43362806528, 67900.2558774827]


In [7]:
# dt and mag data

mag = (1.49972905, 7.42332916, -11.5028191, -4.07344553, -3.28229147, 3.14782126, 3.57746724)
dt =[0.0, 3464.586642261129, 24012.56500505423, 25331.258614414837, 25887.914411486126]
dt_ref = [0.0, 88594.90558064939, 91797.85200701188, 91896.81383935199, 92121.65322824288, 92551.52745506703, 93534.55033071572]

In [8]:
src_guess = (81,71.3)
chi_sq = cluster.chi_squared(src_guess, dt, index = 0)
print(chi_sq)

0.0


In [ ]:
# localization of source position given the index of cluster
x_approx, y_approx, chi_sq = cluster.localize_known_cluster_diffevo(dt_true = dt, index = 1)
print(x_approx, y_approx, chi_sq)
print((x_approx-src_guess[0])**2 + (y_approx-src_guess[1])**2)


In [16]:
# localization of source position with unknown cluster
index, src_pos, chi_sq , src_guess, chi_sqs= cluster.localize_diffevo(dt_true = dt)
print(index, src_pos, chi_sq)

0 [81.00000038017119, 71.29999998618307] 2.1059426626038668e-05


In [ ]:
# localization of source position with unknown cluster for multiple sources
src = pd.read_csv('src_pos_for_distribution.csv')
src_pos_test,index_test = src[['x', 'y']].values, src['indices'].values

for i in range(len(index_test)):
    img = cluster.image_position(src_pos_test[i][0], src_pos_test[i][1], index_test[i])
    dt = cluster.time_delay(img[0], img[1], index_test[i])
    print(f'Finding source {i}')
    index, src_pos, chi_sq , _, _ = cluster.localize_diffevo(dt_true=dt)
    src.at[i, 'localized_index'] = index
    src.at[i, 'localized_x'] = src_pos[0]
    src.at[i, 'localized_y'] = src_pos[1]
    src.at[i, 'localized_chi_sq'] = chi_sq
    src.to_csv('src_pos_for_distribution.csv', index=False)



Finding source 0
